# 3.8.3 Bungee Dunk

---

<br>

*Modeling and Simulation in Python*

Copyright 2021 Allen Downey, (License: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/))

Revised, Mike Augspurger (2021-present)

<br>

---

In [ ]:
#@title
from os.path import basename, exists
from os import mkdir

def download(url,folder):
    filename = folder + basename(url)
    if not exists(folder):
        mkdir(folder)
    # fetches the file at the given url if it is not already present
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/MAugspurger/ModSimPy_MAugs/raw/main/Notebooks/'
        + 'ModSimPy_Functions/modsim.py', 'ModSimPy_Functions/')

from ModSimPy_Functions.modsim import *
import pandas as pd
import numpy as np

In simulating the Penny problem using differential equations and `solve_ivp`, we have made it possible to handle much more complex acceleration problems relatively easily.  

<br>

Why is it easier now? Addition! The Second Law of Motion connects a sum of forces to the second derivative of position (acceleration, that is):

<br>

$$\frac{d^2x}{dt^2} = \frac{\Sigma F}{m}$$

<br>

That sigma $\Sigma$ indicates addition, and this is not hard to handle in a simulation.  We can add whatever forces we'd like to our simulation--even if they change with time or position--and then use the differential equation to determine position and velocity for our object. 

<br>

We'll test this out by dunking a tea biscuit after plummeting down a couple hundred feet.

## Modeling and Implementing the Bungee Dunk

Let's say you want to set the world record for the highest "bungee dunk", which is a stunt in which a bungee jumper dunks a cookie in a cup of tea at the lowest point of a jump. Apparently, this is something you could try [(The video is worth a look)](https://https://www.youtube.com/watch?v=UBf7WC19lpw).

<br>

Since the record is 70 m, let's design a jump for 80 m. We'll start with the following modeling assumptions:

-   Initially the bungee cord hangs from a crane with the attachment point 80 m above a cup of tea.

-   Until the cord is fully extended, it applies no force to the jumper.  (It turns out this might not be a good assumption; but we can always add another force later!)

-   After the cord is fully extended, it obeys Hooke's Law; that is, it applies a force to the jumper proportional to the extension of the cord beyond its resting length: $F = k\Delta x$

-   The mass of the jumper is 75 kg.

-   The jumper is subject to drag force so that their terminal velocity is 70 m/s; this is roughly the terminal velocity of a skydiver who is oriented vertically.

<br>

Our objective is to choose the length of the cord, `L`, and its spring
constant, `k`, so that the jumper falls all the way to the tea cup, but no farther!

### Parameters and the system

We'll create a `params` object to contain the quantities we'll need, including two guesses at the $L$ and $k$:


In [ ]:
params = dict(y_attach = 80.0,   # m,
                 v_init = 0,     # m / s,
                 g = 9.8,        # m/s**2,
                 mass = 75.0,      # kg,
                 area = 0.3,     # m**2 (front facing area of object)
                 rho = 1.2,      # kg/m**3,
                 v_term = 70,    # m / s,
                 L = 25,         # m,
                 k = 40)         # N / m

Now here's a version of `make_system` that takes a `params` object as a parameter.  As in the Penny problem, `make_system` uses the given value of `v_term` to compute the drag coefficient `C_d`.  We'll assume that the jumper jumps from the same height that the cord is attached:

In [ ]:
def make_system(params):
    y_attach, v_init, g, mass, area, rho, v_term, L, k = params.values()
    
    C_d = 2 * mass * g / (rho * area * v_term**2)
    init = pd.Series(dict(y=y_attach, v=v_init))
    t_end = 20

    return dict(y_attach=y_attach, v_init=v_init, g=g,
                mass=mass, area=area, rho=rho,
                v_term=v_term, L=L, k=k,
                C_d=C_d, init=init, t_end=t_end)

In [ ]:
system = make_system(params)
system['C_d']

The coefficient of drag is 0.83, which is a reasonable number given the situation.

### Computing forces

Now we need to calculate the forces on the jumper.  We'll start with `spring_force`, which computes the force of the cord on the jumper.  If the spring is not extended, the spring force is set to 0; if it is extended, the force is calculated using Hooke's Law.

In [ ]:
def spring_force(y, system):
    """Computes the force of the bungee cord on the jumper:
    
    y: height of the jumper
    
    Uses these variables from system|
    y_attach: height of the attachment point
    L: resting length of the cord
    k: spring constant of the cord
    
    returns: force in N
    """
    y_attach, L, k = system['y_attach'], system['L'], system['k']
    
    distance_fallen = y_attach - y
    if distance_fallen <= L:
        return 0
    
    extension = distance_fallen - L
    f_spring = k * extension
    return f_spring

The spring force is 0 until the 25 m cord is fully extended.  When it is extended 1 m, the spring force is 40 N in the positive (upward) direction.

In [ ]:
spring_force(55, system)

In [ ]:
spring_force(54, system)

Now we can compute the drag force as a function of velocity.  Notice how we use the NumPy function `sign()` to make sure that the force is always in the direction opposite to the direction of motion.

In [ ]:
def drag_force(v, system):
    """Computes drag force in the opposite direction of `v`.
    
    v: velocity
    system: System object

    returns: drag force
    """
    rho, C_d, area = system['rho'], system['C_d'], system['area']
    
    f_drag = -np.sign(v) * ((1/2) * rho * v**2 * C_d * area) 
    return f_drag

We can check the internal validity of this function by making sure that when the acceleration created by drag at the terminal velocity is equal and opposite to the acceleration created by gravity.  Here's the drag force at 70 meters per second.

In [ ]:
v = -70
f_drag = drag_force(v, system)
a_drag = f_drag / system['mass']
a_drag

It checks out OK!

### The slope function

Now we can sum our forces in the slope function and determine the overall acceleration using the Second Law and addition:

In [ ]:
def slope_func(t, state, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing g, rho,
            C_d, area, and mass
    
    returns: derivatives of y and v
    """
    y, v = state
    mass, g = system['mass'], system['g']
    
    a_gravity = -g
    a_drag = drag_force(v, system) / mass
    a_spring = spring_force(y, system) / mass
    
    dvdt = a_gravity + a_drag + a_spring
    
    return v, dvdt

As always, let's test the slope function with the initial parameters.  Just as the jumper jumps from the platform, velocity should be zero and the acceleration equal to gravitational acceleration:

In [ ]:
slope_func(0, system['init'], system)

## Analyzing the Bungee Dunk

### Finding the minimum position

Our central aim is to find the $L$ and $k$ values that will allow our jumper to reach 80 m at the bottom of the jump, which would allow her to dunk her tea biscuit.   So let's use `run_solve_ivp` and take a look at the position results:

In [ ]:
results, details = run_solve_ivp(system, slope_func)
results.y.plot(xlabel='Time (s)',
             ylabel='Position (m)');

After reaching the lowest point, the jumper springs back almost to almost 70 m and oscillates several times.  That looks like more oscillation that we expect from an actual jump, which suggests that there is some dissipation of energy in the real world that is not captured in our model.  To improve the model, that might be a good thing to investigate.

<br>

But since we are primarily interested in the initial descent, the model might be good enough for now.  We can use the built-in function `min()` to find the lowest point:

In [ ]:
min(results.y)

At the lowest point, the jumper is still too high, so we'll need to increase `L` or decrease `k`.

### Finding velocity and acceleration

Here's velocity as a function of time:

In [ ]:
results.v.plot(color='C1', label='v', xlabel='Time (s)',
                   ylabel='Velocity (m/s)', legend=True);

Notice that while the oscillation in this plot is similar to that in the position plot, it is shifted in time: when the jumper reaches the nadir at about 5.0 s, the velocity is at 0.0.

<br>  

Acceleration is a bit more complex.  Although we compute acceleration inside the slope function, we don't get acceleration as a result from `run_solve_ivp`.  We can approximate it by computing the numerical derivative of the velocity.  We'll use a NumPy method called `gradient`, and enter the velocity and time sequences as our arguments.  `gradient` produces an array of acceleration values, so to plot it, we match that array up with the time steps in `results.index` to create a `pd.Series`.

In [ ]:
import numpy as np

a = np.gradient(results.v, results.index)
a_series = pd.Series(data=a,index=results.index)
a_series.plot(color='C2',xlabel='Time (s)',
         ylabel='Acceleration (m/$s^2$)');

And we can compute the maximum acceleration the jumper experiences:

In [ ]:
max_acceleration = max(a)
max_acceleration

Relative to the acceleration of gravity, the jumper experiences about 1.7 "g's": that is, he experiences an acceleration that is 1.7 times larger than the acceleration of gravity *g*:

In [ ]:
max_acceleration / system['g']

## Solving for length

Assuming that `k` is fixed, let's find the length `L` that makes the minimum altitude of the jumper exactly 0.

<br>

The metric we are interested in is the lowest point of the first oscillation.  For efficiency, it is better to stop the simulation when we reach this point, rather than run past it and then compute the minimum.

<br>

Here's an event function that stops the simulation when velocity is 0.

In [ ]:
def event_func(t, state, system):
    """Return velocity.
    """
    y, v = state
    return v

As usual, we should test it with the initial conditions.

In [ ]:
event_func(0, system['init'], system)

If we call `run_solve_ivp` with this event function, we'll see that the simulation stops immediately because the initial velocity is 0.

<br>

We can work around that by starting with a very small, non-zero initial velocity.

In [ ]:
params.update(dict(v_init= -0.00001))
system = make_system(params)



Now we can test it and confirm that it stops at the bottom of the jump.

In [ ]:
results, details = run_solve_ivp(system, slope_func, 
                                 events=event_func)
details.message

Here are the results.

In [ ]:
results.y.plot(xlabel='Time (s)',
             ylabel='Position (m)');

And here's the height of the jumper at the lowest point.

In [ ]:
min(results.y)

## Exercises

### Exercise 1

We could find the proper $L$ by the "guess-and-check" method, but we have tools that are more efficient and accurate than that.  In this exercise, use `root_scalar()` to find the optimal length $L$.

<br>  

To do so, follow these steps:
* Write an error function that takes `L` and `system` as arguments, simulates a bungee jump, and returns the lowest point.

* Test the error function with a guess of 25 m and confirm that the return value is about 5 meters.

* Use `root_scalar` with your error function to find the value of `L` that yields a perfect bungee dunk.

* Run a simulation with the result from `root_scalar` and confirm that it works.

It might help to look back at 2.5.1 to remember how we implement `root_scalar`.

In [ ]:
# Write and the error function
# The function should take 'L' and 'system' as parameters
# and should return the final value of y 

def error_func(L, system):


In [ ]:
# Test your error function
# Enter '25' as the 'L' value and make sure it gives you a value
# around 4-5 m



In [ ]:
# Use root_scalar to find the ideal length L
# Your 'bracket' values should be one 'L' value that is too short
# and one that is too long



In [ ]:
# Confirm the results from the previous step
# Update your 'params' using copy() and update(),
# and run the simulation with the new value for 'L'
# You should get a value position value close to 0

